In [14]:
import PyPDF2
import itertools
import spacy
import networkx
from pyvis.network import Network

import pandas as pd
import matplotlib.pyplot as plt

import math

import seaborn as sns
import random
import numpy as np

import re
import sys
sys.path.append("../code/")

from graphUtils import EntityMatcher, makeGraphFromSpacy, from_nx, getFirstOrderGraph, getSecondOrderSubgraph
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get the text from the Mueller Report

In [2]:
graphLocation = 'C:/Users/mj514/Documents/blogdownBlog/blog/static/html/'

In [3]:
mueller = open('../data/Mueller-Report.pdf', 'rb') 
mueller1Reader = PyPDF2.PdfFileReader(mueller)
muellerPages = []
for pageNum in range(mueller1Reader.numPages):
    muellerPages.append([par for par in mueller1Reader.getPage(pageNum).extractText().split('\n')])
mueller.close()
muellerParagraphs = sum(muellerPages,[]) #flatten the list of lists
len(muellerParagraphs)

897

## Prepare Spacy to Extract Entities

In [4]:
nlp = spacy.load('C:/Users/mj514/Anaconda3/lib/site-packages/en_core_web_lg/en_core_web_lg-2.0.0')

In [5]:
terms = (u"collude", u"collusion", 
         u"conspiracy", u"conspire",u"hack", u"hacking", u"cyber intrusions", 
         u"russian hacking", u"hackers",u"social media",u"computer intrusions",
         u"cybersecurity",u"emails","email")
concept_matcher = EntityMatcher(nlp, terms, "THEME")

nlp.add_pipe(concept_matcher, after="ner")

print(nlp.pipe_names)  # The components in the pipeline

doc = nlp(u"This is a hacking, collusion, text about Barack Obama who is the Presedent of The United States")
print([(ent.text, ent.label_) for ent in doc.ents])

['tagger', 'parser', 'ner', 'entity_matcher']
[('hacking', 'THEME'), ('collusion', 'THEME'), ('Barack Obama', 'PERSON'), ('Presedent', 'PERSON'), ('The United States', 'GPE')]


In [6]:
%%time
nlpParagraphs= list(nlp.pipe(muellerParagraphs, n_threads=16, batch_size=50))

Wall time: 44.3 s


In [7]:
%%time
nlpParagraphs= list(nlp.pipe(muellerParagraphs, n_threads=16, batch_size=100))

Wall time: 45.7 s


In [8]:
%%time
nlpParagraphs= list(nlp.pipe(muellerParagraphs, n_threads=8, batch_size=100))

Wall time: 47.3 s


In [9]:
%%time
nlpParagraphs= list(nlp.pipe(muellerParagraphs, n_threads=16, batch_size=1000))

Wall time: 47.4 s


In [10]:
%%time
nlpParagraphs= list(nlp.pipe(muellerParagraphs, n_threads=2, batch_size=1000))

Wall time: 44.7 s


In [11]:
aDoc = nlpParagraphs[105]
aDoc.text

'to release documents directly to reporters and other interested individuals. Specifically, on June 27, 2016, Guccifer 2.0 sent an email to the news outlet The Smoking Gun offering to provide "exclusive access to some leaked emails linked [to] Hillary Clinton\'s staff."148 The GRU later sent the reporter a password and link to a locked portion of the dcleaks.com website that contained an archive of emails stolen by Unit 26165 from a Clinton Campaign volunteer in March 2016.149 That the Guccifer 2.0 persona provided reporters access to a restricted portion of the DCLeaks website tends to indicate that both personas were operated by the same or a closely-related group of people.1so The GRU continued its release efforts through Guccifer 2.0 into August 2016. For example, on August 15, 2016, the Guccifer 2.0 persona sent a candidate for the U.S. Congress documents related to the candidate\'s opponent.1st On August 22, 2016, the Guccifer 2.0 persona transferred approximately 2.5 gigabytes o

In [12]:
muellerParagraphs[105]

'to release documents directly to reporters and other interested individuals. Specifically, on June 27, 2016, Guccifer 2.0 sent an email to the news outlet The Smoking Gun offering to provide "exclusive access to some leaked emails linked [to] Hillary Clinton\'s staff."148 The GRU later sent the reporter a password and link to a locked portion of the dcleaks.com website that contained an archive of emails stolen by Unit 26165 from a Clinton Campaign volunteer in March 2016.149 That the Guccifer 2.0 persona provided reporters access to a restricted portion of the DCLeaks website tends to indicate that both personas were operated by the same or a closely-related group of people.1so The GRU continued its release efforts through Guccifer 2.0 into August 2016. For example, on August 15, 2016, the Guccifer 2.0 persona sent a candidate for the U.S. Congress documents related to the candidate\'s opponent.1st On August 22, 2016, the Guccifer 2.0 persona transferred approximately 2.5 gigabytes o

In [15]:
from collections import Counter
aDoc = nlpParagraphs[105]

entCountDict = Counter([(ent.text, ent.label_) for ent in aDoc.ents if not re.match(r"\.", ent.text) and (ent.label_=='GPE' or ent.label_ =='PERSON' or ent.label_ =='THEME') ])

for k,v in entCountDict.items():
    print(f"Label: {k[1]} text:{k[0]} count:{v}")

Label: PERSON text:Guccifer 2.0 count:3
Label: THEME text:email count:2
Label: THEME text:emails count:5
Label: PERSON text:Hillary Clinton's count:1
Label: PERSON text:GRU count:1
Label: PERSON text:Clinton count:1
Label: GPE text:Florida count:2
Label: PERSON text:DCCC count:2
Label: GPE text:U.S. count:2
Label: PERSON text:Guccifer count:1
Label: PERSON text:repm1er count:1


In [16]:
from spacy import displacy

displacy.serve(aDoc, style="ent")


    Serving on port 5000...
    Using the 'ent' visualizer


    Shutting down server on port 5000.



# Tried using fuzzy wuzzy library to dedup
Wasn't happy with results so I don't use it, but feel free to try it out.

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:

ents = [[ent.text.strip() for ent in doc.ents if not re.match(r"\.", ent.text) 
         and (ent.label_=='GPE' or ent.label_ =='PERSON' or ent.label_ =='THEME')] for doc in nlpParagraphs]
distinctEnts = list(set(sum(ents,[])))

In [ ]:
dedupDict = {}
for i in range(len(distinctEnts)):
    if(i%10 == 0):
        print(len(distinctEnts))
    searchTerm = distinctEnts.pop(0)
    matchList = process.extractBests(searchTerm, distinctEnts[1:],score_cutoff=90)
    
    if(len(matchList) == 0):
        dedupDict[searchTerm] = searchTerm
    else:
        print(searchTerm)
        print(matchList)
        for m in matchList:
            dedupDict[m[0]] = searchTerm
            distinctEnts.remove(m[0])

In [ ]:
def multi_line_toolip(aString: str):
    return(f'<a href="#" title="Line 1&#5;Line 2&#5;Line 3"> {aString}</a>')

## Build the example graph

In [17]:
exampleGraph = Network("400px", "400px", notebook= True)
exampleGraph.add_node('Paragraph', color = 'Grey', title = multi_line_toolip('a pretty really really really really Long String'))
exampleGraph.add_node('Location', color = 'Blue')
exampleGraph.add_node('Person', color = 'Red')
exampleGraph.add_node('Concept/Theme', color = 'Black')

exampleGraph.add_edge('Paragraph','Location')
exampleGraph.add_edge('Paragraph','Person')
exampleGraph.add_edge('Paragraph','Concept/Theme')

#pyvisG.show_buttons(filter_=['physics'])
exampleGraph.save_graph(graphLocation + 'setupGraph.html')
exampleGraph.show('setupGraph.html')

NameError: name 'multi_line_toolip' is not defined

In [18]:
fullMuellerG = makeGraphFromSpacy(nlpParagraphs)
# pyvisG = Network("1000px", "1000px")
# from_nx(pyvisG,fullMuellerG)
# pyvisG.show_buttons(filter_=['physics'])
# #pyvisG.show('fullMueller.html')

In [19]:
fullMuellerG.number_of_nodes()

2780

In [20]:
from networkx.readwrite import json_graph
networkx.write_gpickle(fullMuellerG, "../data/fullMeullerGraph.gpickle")
loadedGraph = networkx.read_gpickle("../data/fullMeullerGraph.gpickle")

In [21]:
pyvisG = Network("1000px", "1000px")
from_nx(pyvisG,loadedGraph)
pyvisG.show_buttons(filter_=['physics'])
pyvisG.show('fullMueller.html')

In [22]:
pyvisG.save_graph(graphLocation + 'fullMuellerGraph.html')

In [23]:
pgRank = networkx.algorithms.pagerank(fullMuellerG)
betweenness = networkx.algorithms.betweenness_centrality(fullMuellerG)
closeness = networkx.algorithms.closeness_centrality(fullMuellerG)

In [171]:


rankings = pd.concat([pd.DataFrame([pgRank]).T,pd.DataFrame([betweenness]).T, pd.DataFrame([closeness]).T],axis = 1)
rankings.columns = ['pgRank','betweenness','closeness']
rankings.sort_values('closeness', ascending = False).head(100).round(4)

,pgRank,betweenness,closeness
Russia,0.0218,0.3379,0.3596
Trump,0.0065,0.1044,0.3157
par_38,0.0008,0.0104,0.3130
par_394,0.0014,0.0080,0.3112
par_251,0.0015,0.0141,0.3104
par_375,0.0012,0.0112,0.3097
par_205,0.0019,0.0100,0.3068
par_372,0.0007,0.0068,0.3066
par_461,0.0015,0.0099,0.3064
par_180,0.0021,0.0160,0.3061


In [25]:
muellerParagraphs[394]

'U.S. Department of Justice Attorney 1Nork Prodttet II :May Contain Ma:teria:l Preteeted Under Fed. R. Criffl. P. 6(e) Second, Flynn made false statements about calls he had previously made to representatives of Russia and other countries regarding a resolution submitted by Egypt to the United Nations Security Council on December 21, 2016. Specifically, Flynn stated that he only asked the countries\' positions on how they would vote on the resolution and that he did not request that any of the countries take any particular action on the resolution. That statement was false. On December 22, 2016, Flynn called Kislyak, informed him of the incoming Trump Administration\'s opposition to the resolution, and requested that Russia vote against or delay the resolution. Flynn also falsely stated that Kislyak never described Russia\'s response to his December 22 request regarding the resolution. Kislyak in fact told Flynn in a conversation on December 23, 2016, that Russia would not vote against

In [26]:
from spacy.attrs import ORTH
wordCountDict = {id: len(doc.count_by(ORTH)) for id, doc in enumerate(nlpParagraphs)}
pd.DataFrame.from_records(wordCountDict, ['count']).T.sort_values('count', ascending = False)

,count
771,372
764,360
767,355
770,353
791,349
252,346
800,342
684,340
441,338
428,337


In [31]:
parExample = getFirstOrderGraph(fullMuellerG,'par_415')
pyvisG = Network("500px", "500px")
from_nx(pyvisG,parExample)
pyvisG.show('par_415.html')


Nodes: 11
Edges: 10
Self Loops: 0


In [32]:
subGraph

NameError: name 'subGraph' is not defined

In [33]:
muellerParagraphs[69]

'.S. operations included social media specialists focusing on Facebook, YouTube, and Twitter.43 The IRA later added specialists who operated on Tumblr and Instagram accounts.44 Initially, the IRA created social media accounts that pretended to be the personal accounts of U.S. persons.45 By early 2015, the IRA began to create larger social media groups or public social media pages that claimed (falsely) to be affiliated with U.S. political and grassroots organizations. In certain cases, the IRA created accounts that mimicked real U.S. organizations. For example, one IRA-controlled Twitter account, @TEN_ GOP, purported to be connected to the Tennessee Republican Party.46 More commonly, the IRA created accounts in the names of fictitious U.S. organizations and grassroots groups and used these accounts to pose as immigration groups, Tea Party activists, Black Lives Matter protestors, and other U.S. social and political activists. Harm to Ongoing Matter Harm to Ongoing Matter Harm to Ongoin

In [34]:
parExample = getSecondOrderSubgraph(fullMuellerG,'Ivanka Trump')
ivankaViz = Network("500px", "500px")
from_nx(ivankaViz,parExample)
ivankaViz.show('Ivanka.html')
ivankaViz.save_graph(graphLocation + 'ivankaViz.html')

Nodes: 58
Edges: 68
Self Loops: 0


In [ ]:
conway = getSecondOrderSubgraph(fullMuellerG,'Kellyanne Conway')
conwayViz = Network("500px", "500px")
from_nx(conwayViz,conway)
conwayViz.show('conway.html')
conwayViz.save_graph(graphLocation + 'conwayViz.html')

In [ ]:
conwayViz.save_graph(graphLocation + 'conwayViz.html')

In [ ]:
comeySubgraph = getSecondOrderSubgraph(fullMuellerG,'Corney')
comeyViz = Network("500px", "500px")
from_nx(comeyViz,comeySubgraph)
comeyViz.show('comeyViz.html')
comeyViz.save_graph(graphLocation + 'comeyViz.html')

In [ ]:
comeySubgraph = getSecondOrderSubgraph(fullMuellerG,'Corney')

closeness = networkx.algorithms.betweenness_centrality(comeySubgraph)
pd.DataFrame([closeness]).T.sort_values(0, ascending=False)

In [132]:
paragraphDF = pd.DataFrame({'text':muellerParagraphs,'paragraph':[f'par_{num}' for num in range(len(muellerParagraphs))]})
paragraphDF['paragraph'] = paragraphDF.paragraph.astype('str') # Because pandas is stupid
def whatToRead(concept, fullGraph, topn = 10):
    subGraph = getSecondOrderSubgraph(fullGraph,concept)
    closeness = networkx.algorithms.betweenness_centrality(subGraph)
    readDF = pd.DataFrame([closeness]).T.sort_values(0, ascending=False).head(topn)
    readDF.reset_index(inplace=True)
    print(readDF)
    readDF.columns = ['paragraph','score']
    matchingParagraphs = readDF.merge(paragraphDF,on = 'paragraph', how = 'inner')
    return matchingParagraphs

In [133]:
toRead = whatToRead('Kilimnik',fullMuellerG)

Nodes: 129
Edges: 220
Self Loops: 0
      index         0
0  Kilimnik  0.232883
1  par_294   0.181871
2  par_292   0.168380
3  par_285   0.162705
4  Manafort  0.152514
5  par_293   0.150722
6  par_291   0.143568
7  par_281   0.140322
8  Ukraine   0.123043
9  par_278   0.116656


In [170]:
toRead = whatToRead('Trump',fullMuellerG)

Nodes: 854
Edges: 1686
Self Loops: 0
     index         0
0  Trump    0.594863
1  Russia   0.134729
2  par_831  0.064194
3  par_842  0.050346
4  par_826  0.046688
5  par_180  0.038710
6  par_843  0.037992
7  par_252  0.034149
8  par_818  0.034117
9  par_453  0.030026


In [ ]:
ConwaySubgraph = getSecondOrderSubgraph(fullMuellerG,'Kellyanne Conway')
ConwayViz = Network("500px", "500px")
from_nx(ConwayViz,ConwaySubgraph)
ConwayViz.show('ConwayViz.html')
ConwayViz.save_graph(graphLocation + 'ConwayViz.html')

In [ ]:
toRead.reset_index(inplace=True)
toRead.columns = ['paragraph','score']

In [ ]:
toRead.columns

In [ ]:
toRead.reset_index( inplace=True)

In [134]:
paragraphDF[paragraphDF.paragraph == 'par_294']

,text,paragraph
294,"U.S. Department of Justice Atterney Werk Predttet // May Centain Material Preteeted Under Fed. R. Crim. P. 6(e) first meeting, which took place in Madrid, Spain in January 2017, was with Georgiy Oganov. Oganov, who had previously worked at the Russian Embassy in the United States, was a senior executive at a Deripaska company and was believed to report directly to Deripaska.940 Manafort initially denied attending the meeting. When he later acknowledged it, he claimed that the meeting had been arranged by his lawyers and concerned only the Pericles lawsuit.941 Other evidence, however, provides reason to doubt Manafort's statement that the sole topic of the meeting was the Pericles lawsuit. In particular, text messages to Manafort from a number associated with Kilimnik suggest that Kilimnik and Boyarkin-not Manafort's counsel-had arranged the meeting between Manafort and Oganov.942 Kilimnik's message states that the meeting was supposed to be ""not about money or Pericles"" but instead ""about recreating [the] old friendship""-ostensibly between Manafort and Deripaska-""and talking about global politics.""943 Manafort also replied by text that he ""need[s] this finished before Jan. 20,""944 which appears to be a reference to resolving Pericles before the inauguration. On January 15, 2017, three days after his return from Madrid, Manafort emailed K.T. McFarland, who was at that time designated to be Deputy National Security Advisor and was formally appointed to that position on January 20, 2017.945 Manafort's January 15 email to McFarland stated: ""I have some important information I want to share that I picked up on my travels over the last month.""946 Manafort told the Office that the email referred to an issue regarding Cuba, not Russia or Ukraine, and Manafort had traveled to Cuba in the past month.947 Either way, McFarland-who was advised by Flynn not to respond to the Manafort appears not to have responded to Manafort. 948 Manafort told the Office that around the time of the Presidential Inauguration in January, he met with Kilimnik and Ukrainian oligarch Serhiy Lyovochkin at the Westin Hotel in Alexandria, Virginia.949 During this meeting, Kilimnik again discussed the Y anukovych peace plan that he had broached at the August 2 meeting and in a detailed December 8, 2016 message found in Kilimnik' s DMP email account.950 In that December 8 email, which Manafort 94° Kalashnikova 5/17/18 302, at 4; Gary Lee, Soviet Embassy's Identity Crisis, Washington Post (Dec. 20, 1991); Georgy S. Oganov Executive Profile & Biography, Bloomberg (Mar. 12, 2019). 941 Manafort 9/11/18 302, at 7. 942 Text Message, Manafort & Kilimnik. 943 Text Message, Manafort & Kilimnik; Manafort 9/12/18 302, at 5. 944 Text Message, Manafort & Kilimnik. 945 1/15/17 Email, Manafort, McFarland, & Flynn. 946 1/15/17 Email, Manafort, McFarland, & Flynn. 947 Manafort 9/11/18 302, at 7. 948 1 /15/17 Email, Manafort, McFarland, & Flynn; McFarland 12/22/17 302, at 18-19. 949 Manafort 9/11/18 302, at 7; Manafort 9/21/18 302, at 3; 1/19/17 & 1/22/17 Kilimnik CBP Records, Jan. 19 and 22, 2017; 2016-17 Text Messages, Kilimnik & Patten, at 1-2. 950 Investigative Technique 142",par_294


In [ ]:
paragraphDF

In [ ]:
pd.options.display.max_colwidth = -1
toRead.merge(paragraphDF,on = 'paragraph', how = 'inner').tail()

In [ ]:
muellerParagraphs[461]

# Node2Vec

In [49]:
nxG = fullMuellerG

startingNodes = nxG.nodes()
numWalks = 100000

walkLengths = np.random.normal(15, 3, numWalks).astype(int)
walkLengths[walkLengths < 0] = 3 # has to do three steps at least

walks = []
for walkLen in walkLengths:
    start = random.choice(list(startingNodes))# Start at a random node
    if nxG.degree(start) > 1: #Check if the starting node has neighbors
        walk = [start] #Start the walk
        for step in range(walkLen):
            paths = [n for n in nxG.neighbors(walk[-1])] #Get a list of edges to walk along
            walk.append(random.choice(paths)) #pick one and walk walk that way
        walks.append(walk)

In [50]:
walks[3]

['Ukraine',
 'par_292',
 'email',
 'par_126',
 'Attort1ey',
 'par_126',
 'Russia',
 'par_279',
 'Viktor Yanukovych',
 'par_891',
 'Konstantin Kilimnik',
 'par_340',
 'Konstantin Kilimnik',
 'par_39',
 'Konstantin Kilimnik',
 'par_838']

In [51]:
randomWalk = fullMuellerG.subgraph(walks[3])
randomVis = Network("450px", "450px", notebook=True)
from_nx(randomVis,randomWalk)
#pyvisG.show_buttons(filter_=['physics'])
randomVis.show('randomWalk.html')

#randomVis.save_graph(graphLocation + 'randomVis.html')

In [52]:
randomVis.save_graph(graphLocation + 'randomVis.html')

In [53]:
randomWalk2 = fullMuellerG.subgraph(walks[6001])
randomVis2 = Network("500px", "500px", notebook=True)
from_nx(randomVis2,randomWalk2)
#pyvisG.show_buttons(filter_=['physics'])
randomVis2.show('randomWalk2.html')

In [54]:
from gensim.models import Word2Vec
w2vModel = Word2Vec(walks, size=128, window=10, min_count=1, workers=16)

In [55]:
w2vModel.wv['par_459']

array([ 8.5658211e-01, -7.5977959e-02,  1.9293311e-01,  2.8880692e-01,
       -3.7974787e-01,  1.7113903e+00, -1.6320123e-01,  6.7774665e-01,
       -3.6900899e-01,  1.5136051e+00, -9.0886843e-01,  3.7862536e-01,
       -1.4933616e+00, -4.9000210e-01, -1.1703711e-01, -1.7433348e+00,
        1.2359954e+00, -5.8149803e-01, -3.4730569e-01,  7.0207730e-02,
       -5.8888537e-01,  1.1004691e+00,  4.4984561e-01, -1.8141411e-01,
       -1.0406804e+00,  6.5777808e-01,  2.7915001e-01, -2.1235332e+00,
        4.5746171e-01,  7.1488458e-01,  9.4297409e-01,  1.1773700e+00,
        4.9657321e-01,  1.3415179e+00, -4.8740792e-01,  3.6713502e-01,
        9.9048629e-02, -1.0810277e+00, -1.1509949e+00,  1.2439925e-01,
       -1.0167325e-03,  5.8631516e-01,  3.2439262e-01, -6.0114014e-01,
       -4.9349684e-02,  3.9286095e-01,  8.8976824e-01,  5.8819842e-01,
       -5.8601880e-01, -9.1387260e-01, -9.7754496e-01,  1.1068614e+00,
        2.1339642e-01,  5.5767983e-01,  3.8735968e-01,  1.1427204e-01,
      

In [56]:
w2vModel.wv.most_similar('par_459', topn = 20)

[('Michael Flynn Overview', 0.7719929814338684),
 ('Michael Flynn', 0.6361454725265503),
 ('Mtty', 0.5839493274688721),
 ('par_890', 0.5175340175628662),
 ('par_343', 0.5146706700325012),
 ('par_414', 0.5072144269943237),
 ('par_32', 0.4995189309120178),
 ('par_866', 0.4674863815307617),
 ('WmŁk Pfoe', 0.46587827801704407),
 ('par_415', 0.46530914306640625),
 ('par_852', 0.460650771856308),
 ('par_347', 0.44727665185928345),
 ('par_865', 0.44042789936065674),
 ('par_502', 0.4363621473312378),
 ('par_22', 0.42809543013572693),
 ('par_77', 0.424285888671875),
 ('par_393', 0.4234082102775574),
 ('par_677', 0.4189611077308655),
 ('par_434', 0.41543346643447876),
 ('Michael Flynn US', 0.41116708517074585)]

In [57]:
w2vModel.wv.most_similar(positive=['Russia', 'the District of Columbia'], negative=['Moscow'])

[('par_892', 0.5586262941360474),
 ('par_355', 0.4765755534172058),
 ('Virginia', 0.45990994572639465),
 ('par_356', 0.4562985301017761),
 ('par_280', 0.4541923701763153),
 ('par_274', 0.450406551361084),
 ('par_890', 0.44122016429901123),
 ('par_679', 0.4333244562149048),
 ('Davis', 0.4312102496623993),
 ('par_681', 0.42714083194732666)]

In [58]:
w2vModel.wv.most_similar(positive=['Russia', 'the United States'], negative=['Putin'])

[('par_343', 0.5213621854782104),
 ('par_367', 0.49795305728912354),
 ('par_276', 0.4262751340866089),
 ('par_78', 0.4103592038154602),
 ('par_273', 0.40767204761505127),
 ('par_459', 0.3821890354156494),
 ('New Zealand', 0.38109439611434937),
 ('par_41', 0.3798227906227112),
 ('par_355', 0.37040936946868896),
 ('conspiracy', 0.36254751682281494)]

In [59]:
w2vModel.wv.most_similar(positive=['Clinton', 'Trump'], negative=['email'])

[('par_582', 0.4163612127304077),
 ('Conway et al', 0.41269049048423767),
 ('par_142', 0.40394359827041626),
 ('par_34', 0.37245500087738037),
 ('SCR015_000175', 0.3654935956001282),
 ('par_240', 0.3582696318626404),
 ('par_465', 0.35356855392456055),
 ('Natalia Veselnitskaya', 0.348793625831604),
 ('par_73', 0.34508466720581055),
 ('Veselnitskaya', 0.3378516435623169)]

In [60]:
w2vModel.wv.most_similar('McGahn', topn=20)

[('par_668', 0.7105333805084229),
 ('par_477', 0.6837894916534424),
 ('\\1/erk PreElttet', 0.6784788370132446),
 ('par_512', 0.6711134910583496),
 ('par_656', 0.6684775352478027),
 ('par_598', 0.6600894927978516),
 ('par_663', 0.6596580743789673),
 ('par_618', 0.6512137651443481),
 ('par_662', 0.6500329375267029),
 ('par_672', 0.6448381543159485),
 ('par_541', 0.632118821144104),
 ('SCR015_000198', 0.6310539245605469),
 ('par_596', 0.6307615041732788),
 ('May Cattta', 0.621712327003479),
 ('par_584', 0.6211329698562622),
 ('par_604', 0.6205765604972839),
 ('par_660', 0.6180634498596191),
 ('SCR0l5_000278', 0.6176866292953491),
 ('par_532', 0.6134857535362244),
 ('Call Rod', 0.6130787134170532)]

In [61]:
muellerParagraphs[34]

"U.S. Department of Justice Atterttey 'Nerk Prndttet // Mtty Cetttttitt Moterisl Preteeted Uttder Fed. R. Crim. P. 6(e) National Security Agency-that concluded with high confidence that Russia had intervened in the election through a variety of means to assist Trump's candidacy and harm Clinton's. A declassified version of the assessment was publicly released that same day. Between mid-January 2017 and early February 2017, three congressional committees-the House Permanent Select Committee on Intelligence (HPSCI), the Senate Select Committee on Intelligence (SSCI), and the Senate Judiciary Committee (SJC)-announced that they would conduct inquiries, or had already been conducting inquiries, into Russian interference in the election. Then-FBI Director James Corney later confirmed to Congress the existence of the FBI's investigation into Russian interference that had begun before the election. On March 20, 2017, in open-session testimony before HPSCI, Corney stated: I have been authorize

In [62]:
muellerParagraphs[343]

'Malaysia, New Zealand, Senegal, and Venezuela resubmitted the resolution.1218 Throughout the day, members of the Transition Team continued to talk with foreign leaders about the resolution, with Flynn continuing to lead the outreach with the Russian government through Kislyak.1219 When Flynn again spoke with Kislyak, Kislyak informed Flynn that if the resolution came to a vote, Russia would not vote against it. 1220 The resolution later passed 14-0, with the United States '

# Lets do Interesting Things With Our Representation

In [68]:
paragraphDF = pd.DataFrame({'text':muellerParagraphs,'paragraph':[f'par_{num}' for num in range(len(muellerParagraphs))]})

In [69]:
par = 'par_390'

def showMoreLikeThis(par):
    mostSimilar = w2vModel.wv.most_similar(par, topn=20)

    mostSimilarParagraphs = [simTuple for simTuple in mostSimilar if simTuple[0].startswith('par')]
    similarDF = pd.DataFrame(mostSimilarParagraphs, columns=['paragraph','similarity'])
    showDF = similarDF.merge(paragraphDF, on = 'paragraph')
    return showDF

In [72]:
showMoreLikeThis('par_124')

,paragraph,similarity,text
0,par_123,0.840400,"eleasing. The file-transfer evidence described above and other information uncovered during the investigation discredit WikiLeaks's claims about the source of material that it posted. Beginning in the summer of 2016, Assange and WikiLeaks made a number of statements about Seth Rich, a former DNC staff member who was killed in July 2016. The statements about Rich implied falsely that he had been the source of the s"
1,par_139,0.794321,"U.S. Department of Justice Attet'He~· Werk Prnfitiet // Mft)· CemaiH Material Preteetee UHeer Fee. R. Criffl. P. 6(e) enormous and recalled telling the conference call the Access Hollywood tape was coming.245 Corsi stated that he was convinced that his efforts had caused WikiLeaks to release the emails when they did.246 In a later November 2018 interview, Corsi stated that he thought that he had told people on a WND conference call about the forthcoming tape and had sent out a tweet asking whether anyone could contact Assange, but then said that maybe he had done nothing.247 The Office investigated Corsi' s allegations about the events of October 7 little corroboration for his alle ations about the da .248 However, the phone records themselves do not indicate that the conversation was with any of the reporters who broke the Access Hollywood sto , and the Office has not otherwise been able to identif the substance of the conversation. However, the Office has not identified any conference call participant, or anyone who spoke to Corsi that day, who says that they received non-public information about the tape from Corsi or acknowledged having contacted a member of WikiLeaks on October 7, 2016 after a conversation with Corsi. e. Donald Trump Jr. Interaction with WikiLeaks Donald Trump Jr. had direct electronic communications with WikiLeaks during the campaign period. On September 20, 2016, an individual named Jason Fishbein sent WikiLeaks the password for an unlaunched website focused on Trump's ""unprecedented and dangerous"" ties 245 During the same interview, Corsi also suggested that he may have sent out public tweets because he knew Assange was reading his tweets. Our Office was unable to find evidence of any such tweets. 246_ Corsi 9/21/18 302, at 6-7. 247 Corsi 11/1/18 302, at 6. Harm to Ongoing Matter ··Harm to Ongoing Matter Harm to Ongoing Matter 59"
2,par_138,0.727800,"U.S. Department of Justice AM:erttey Wer:k Predttet // Mtty Cetttaitt Mttterial Preteeted Uttder FeE:I. R. Criffi. P. 6(e) d. WikiLeaks's October 7, 2016 Release of Stolen Podesta Emails On October 7 2016 four days after the Assange press conference , the Washington Post published an Access Hollywood video that captured comments by candidate Trump some years earlier and that was expected to adversely affect the Campaign.239 Less than an hour after the video's publication, WikiLeaks released the first set of emails stolen by the GRU from the account of Clinton Campaign chairman John Podesta. Harm to Ongoing Matter 111Harm to Ongoing Matter -Harm to Ongoing Matter Harm to Ongoing Matter 1111Harm to Ongoing Matter Corsi said that, because he had no direct means o communicating with WikiLeaks, he told members of the news site WNO-who were participating on a conference call with him that day-to reach Assange immediately.244 Corsi claimed that the pressure was 239 Candidate Trump can be heard off camera making graphic statements about women. 240 241 242 243 244 In a later November 2018 interview, Corsi stated Harm to Ongoing Matter that he believed Malloch was on the call but then focused on other individuals who were on the call-invitation, which Malloch was not. (Separate travel records show that at the time of the call, Malloch was aboard a transatlantic flight). Corsi at one point stated that after WikiLeaks 's release of stolen emails on October 7, 2016, he concluded Malloch had gotten in contact with Assange. Corsi 11/1/18 302, at 6. 58"
3,par_135,0.706262,"U.S. Department of J

In [ ]:
par_124

In [98]:
# Flatten the graph. Should have had this data structure come out when we were building the graph.

MuellerList = list(fullMuellerG.nodes(data=True))
flat_list = []
for node_item in MuellerList:
    #print(node_item)
    label = node_item[0]
    if label.startswith('par'):
        text = node_item[1]['text']
        d_type = node_item[1]['t']
    else:
        text = label
        d_type = node_item[1]['label']
    flat_list.append({'Label':label,'Text':text,'D_type':d_type})
MuellerDF = pd.DataFrame(flat_list)

In [103]:
MuellerDF.D_type.value_counts()

PERSON       1702
Paragraph    809 
GPE          258 
THEME        11  
Name: D_type, dtype: int64

In [119]:
def findRelatedConcepts(ent, target_type):
    mostSimilar = w2vModel.wv.most_similar(ent, topn=100)

    #mostSimilarParagraphs = [simTuple for simTuple in mostSimilar if simTuple[0].startswith('par')]
    similarDF = pd.DataFrame(mostSimilar, columns=['Label','similarity'])
    showDF = similarDF.merge(MuellerDF, on = 'Label')
    return showDF[showDF['D_type'] == target_type].head(5)

In [129]:
findRelatedConcepts('Kilimnik','GPE')

,Label,similarity,D_type,Text
0,Kiev,0.658731,GPE,Kiev
16,New York City,0.549959,GPE,New York City
17,Madrid,0.549046,GPE,Madrid
18,Ukraine,0.548903,GPE,Ukraine
22,Donbas,0.533154,GPE,Donbas


In [130]:
findRelatedConcepts('Kilimnik','PERSON')

,Label,similarity,D_type,Text
2,Deripaska,0.639387,PERSON,Deripaska
8,Yanukovych,0.599383,PERSON,Yanukovych
10,Pericles,0.589324,PERSON,Pericles
12,Fabrizio,0.582167,PERSON,Fabrizio
14,Kasanofto Kilimnik,0.568608,PERSON,Kasanofto Kilimnik


In [125]:
findRelatedConcepts('Kilimnik','Paragraph')

,Label,similarity,D_type,Text
1,par_290,0.653940,Paragraph,"Russian officials.906 Kilimnik then traveled to Washington, D.C. on or about May 5, 2016; while in Washington, Kilimnik had pre-arranged meetings with State Department employees.907 Late on the evening of May 6, Gates arranged for Kilimnik to take a 3:00 a.m. train to meet Mana fort in New York for breakfast on May 7. 908 According to Manafort, during the meeting, he and Kilimnik talked about events in Ukraine, and Manafort briefed Kilimnik on the Trump Campaign, expecting Kilimnik to pass the information back to individuals in Ukraine and elsewhere.909 Manafort stated that Opposition Bloc members recognized Manafort's position on the Campaign was an opportunity, but Kilimnik did not ask for anything.91° Kilimnik spoke about a plan of Boyko to boost election participation in the eastern zone of Ukraine, which was the base for the Opposition Bloc.911 Kilimnik returned to Washington, D.C. right after the meeting with Manafort. Manafort met with Kilimnik a second time at the Grand Havana Club in New York City on the evening of August 2, 2016. The events leading to the meeting are as follows. On July 28, 2016, Kilimnik flew from Kiev to Moscow.912 The next day, Kilimnik wrote to Manafort requesting that they meet, using coded language about a conversation he had that day.913 In an email with a subject line ""Black Caviar,"" Kilimnik wrote: I met today with the guy who gave you your biggest black caviar jar several years ago. We spent about 5 hours talking about his story, and I have several important messages from him to you. He asked me to go and brief you on our conversation. I said I have to run it by you first, but in principle I am prepared to do it. ... It has to do about the future of his 905 Investigative Technique 906 4/26/16 Email, Kilimnik to Purcell, at 2; Gates 2/2/18 302, at 12; Patten 5/22/18 302, at 6-7; Gates 11/7/18 302, at 3. 907 5/7/16 Email, Kilimnik to Charap & Kimmage; 5/7/16 Email, Kasanofto Kilimnik. 908 5/6/16 Email, Manafort to Gates; 5/6/16 Email, Gates to Kilimnik. 909 Manafort 10/11/18 302, at 1. 910 Manafort 10/11/18 302, at 1. 911 Manafort 10/11/18 302, at 1. 912 7/25/16 Email, Kilimnik to katrin@yana.kiev.ua (2: 17:34 a.m.). 913 7/29/16 Email, Kilimnik to Manafort (10:51 a.111.). 138"
3,par_291,0.629404,Paragraph,"U.S. Department of Justice Attorttey Work Prodttet // Mtt, CoHtttiH Mttterittl Proteeted Uttder Fed. R. Criffl. P. 6(e) country, and is quite interesting.914 Manafort identified ""the guy who gave you your biggest black caviar jar"" as Yanukovych. He explained that, in 2010, he and Y anukovych had lunch to celebrate the recent presidential election. Yanukovych gave Manafort a large jar of black caviar that was worth approximately $30,000 to $40,000.915 Manafort's identification ofYanukovych as ""the guy who gave you your biggest black caviar jar"" is consistent with Kilimnik being in Moscow-where Yanukovych resided-when Kilimnik wrote ""I met today with~ a December 2016 email in which Kilimnik referred to Yanukovych as ""BG,""-916 Manafort replied to Kilimnik's July 29 email, ""Tuesday [August 2] is best ... Tues or weds in NYC.""917 Three days later, on July 31, 2016, Kilimnik flew back to Kiev from Moscow, and on that same day, wrote to Manafort that he needed ""about 2 hours"" for their meeting ""because it is a long caviar story to tell.""918 Kilimnik wrote that he would arrive at JFK on August 2 at 7:30 p.m., and he and Manafort agreed to a late dinner that night.919 Documentary evidence-including flight, phone, and hotel records, and the timing of text messages exchanged920-confirms the dinner took place as planned on August 2.921 As to the contents of the meeting itself, the accounts of Manafort and Gates-who arrived late to the dinner-differ in certain respects. But their versions of events, when assessed alongside available documentary evidence and what Kilimnik told business associate Sam Patten, indicate that at least 

In [165]:
pd.options.display.max_rows = 400
MuellerDF[MuellerDF.D_type=='GPE'].Text.value_counts()

Prikhodko                                            1
Gauhar-000057                                        1
Syria                                                1
Klimentov                                            1
IO                                                   1
Kremlin                                              1
See United States                                    1
Prntittet                                            1
EGYPT                                                1
te!'fl                                               1
Interfax-Ukraine                                     1
iA                                                   1
Evgeney                                              1
Singapore                                            1
New York City                                        1
Washington                                           1
D]uring                                              1
Ukraine                                              1
Protlttet 

In [163]:
findRelatedConcepts('emails','PERSON')

,Label,similarity,D_type,Text
3,Clinton,0.530507,PERSON,Clinton
10,Seth Rich's,0.485585,PERSON,Seth Rich's
11,GRU,0.479961,PERSON,GRU
14,SM-2589105-DCLEAKS,0.477812,PERSON,SM-2589105-DCLEAKS
18,John Podesta,0.463893,PERSON,John Podesta


In [204]:
findRelatedConcepts('hacking','PERSON')

,Label,similarity,D_type,Text
10,DCCC,0.535998,PERSON,DCCC
16,Aleksei Sergeyevich Morenets,0.474751,PERSON,Aleksei Sergeyevich Morenets
19,Asofthiswriting,0.457502,PERSON,Asofthiswriting
21,Podesta,0.437288,PERSON,Podesta
23,Hilar,0.431587,PERSON,Hilar
